# <center> Идентификация пользователей по посещенным веб-страницам

В этом проекте мы будем решать задачу идентификации пользователя по его поведению в сети Интернет. Это сложная и интересная задача на стыке анализа данных и поведенческой психологии. В качестве примера, компания Яндекс решает задачу идентификации взломщика почтового ящика по его поведению. В двух словах, взломщик будет себя вести не так, как владелец ящика: он может не удалять сообщения сразу по прочтении, как это делал хозяин, он будет по-другому ставить флажки сообщениям и даже по-своему двигать мышкой. Тогда такого злоумышленника можно идентифицировать и "выкинуть" из почтового ящика, предложив хозяину войти по SMS-коду. Этот пилотный проект описан в [статье](https://habrahabr.ru/company/yandex/blog/230583/) на Хабрахабре. Похожие вещи делаются, например, в Google Analytics и описываются в научных статьях, найти можно многое по фразам "Traversal Pattern Mining" и "Sequential Pattern Mining".

<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

Мы будем решать похожую задачу: по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком, мы будем идентифицировать этого человека. Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

Будем использовать данные из [статьи](http://ceur-ws.org/Vol-1058/) "A Tool for Classification of Sequential Data". И хотя мы не можем рекомендовать эту статью (описанные методы делеки от state-of-the-art, лучше обращаться к [книге](http://www.charuaggarwal.net/freqbook.pdf) "Frequent Pattern Mining" и последним статьям с ICDM), но данные там собраны аккуратно и представляют интерес.

Данные собраны с прокси-серверов Университета Блеза Паскаля и имеют очень простой вид: <br>

<center>ID пользователя, timestamp, посещенный веб-сайт</center>

Скачать исходные данные можно по [ссылке](http://fc.isima.fr/~kahngi/cez13.zip) в статье, там же описание.
Для этого задания хватит данных не по всем 3000 пользователям, а по 10 и 150. [Ссылка](https://yadi.sk/d/_HK76ZDo32AvNZ) на архив *capstone_websites_data.zip* (~7.1 Mb, в развернутом виде ~ 65 Mb). 

В финальном проекте уже придется столкнуться с тем, что не все операции можно выполнить за разумное время (скажем, перебрать с кросс-валидацией 100 комбинаций параметров случайного леса на этих данных Вы вряд ли сможете), поэтому мы будем использовать параллельно 2 выборки: по 10 пользователям и по 150. Для 10 пользователей будем писать и отлаживать код, для 150 – будет рабочая версия. 

Данные устроены следующем образом:

 - В каталоге 10users лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя;
 - Аналогично для каталога 150users – там 150 файлов;
 - В 3users_toy – игрушечный пример из 3 файлов, это для отладки кода предобработки, который Вы далее напишете.

# <center>Неделя 1. Подготовка данных к анализу и построению моделей

Первая часть проекта посвящена подготовке данных для дальнейшего описательного анализа и построения прогнозных моделей. Надо будет написать код для предобработки данных (исходно посещенные веб-сайты указаны для каждого пользователя в отдельном файле) и формирования единой обучающей выборки. Также в этой части мы познакомимся с разреженным форматом данных (матрицы Scipy.sparse), который хорошо подходит для данной задачи. 

**План 1 недели:**
 - Часть 1. Подготовка обучающей выборки
 - Часть 2. Работа с разреженным форматом данных

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций 1 и 2 недели курса "Математика и Python для анализа данных":**
   - [Циклы, функции, генераторы, list comprehension](https://www.coursera.org/learn/mathematics-and-python/lecture/Kd7dL/tsikly-funktsii-ghienieratory-list-comprehension)
   - [Чтение данных из файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/8Xvwp/chtieniie-dannykh-iz-failov)
   - [Запись файлов, изменение файлов](https://www.coursera.org/learn/mathematics-and-python/lecture/vde7k/zapis-failov-izmienieniie-failov)
   - [Pandas.DataFrame](https://www.coursera.org/learn/mathematics-and-python/lecture/rcjAW/pandas-data-frame)
   - [Pandas. Индексация и селекция](https://www.coursera.org/learn/mathematics-and-python/lecture/lsXAR/pandas-indieksatsiia-i-sieliektsiia)
   
**Кроме того, в задании будут использоваться библиотеки Python [glob](https://docs.python.org/3/library/glob.html), [pickle](https://docs.python.org/2/library/pickle.html) и класс [csr_matrix](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html) из scipy.sparse.**

Наконец, для лучшей воспроизводимости результатов приведем список версий основных используемых в проекте библиотек: NumPy, SciPy, Pandas, Matplotlib, Statsmodels и Scikit-learn. Для этого воспользуемся расширением [watermark](https://github.com/rasbt/watermark). Я использую Anaconda3 версии 4.2.0.

In [1]:
# pip install watermark
%load_ext watermark

In [2]:
#%watermark?

In [3]:
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,scikit-learn

CPython 2.7.12
IPython 4.2.0

numpy 1.11.2
scipy 0.18.1
pandas 0.18.1
matplotlib 1.5.1
statsmodels 0.8.0.dev0+302c2ec
scikit-learn 0.18.1

compiler   : MSC v.1500 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 42 Stepping 7, GenuineIntel
CPU cores  : 2
interpreter: 64bit


C:\Users\User\Anaconda2\lib\site-packages\watermark\watermark.py:143: DeprecationWarning: Importing scikit-learn as `scikit-learn` has been depracated and will not be supported anymore in v1.7.0. Please use the package name `sklearn` instead.
  DeprecationWarning)


In [4]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import pickle

**Посмотрим на один из файлов с данными о посещенных пользователем (номер 31) веб-страницах.**

In [5]:
user31_data = pd.read_csv('capstone_websites_data/10users/user0031.csv', header=None,
                  names=['id', 'timestamp', 'site'])

In [6]:
user31_data.head()

,id,timestamp,site
0,31,2013-11-15T08:17:10,api.dailymotion.com
1,31,2013-11-15T08:21:43,b12.myspace.com
2,31,2013-11-15T08:21:46,b12.myspace.com
3,31,2013-11-15T08:22:14,www.facebook.com
4,31,2013-11-15T08:37:30,pixel2368.everesttech.net


**Поставим задачу классификации: идентифицировать пользователя по сессии из 10 подряд посещенных сайтов. Объектом в этой задаче будет сессия из 10 сайтов, последовательно посещенных одним и тем же пользователем, признаками – индексы этих 10 сайтов (чуть позже здесь появится "мешок" сайтов, подход Bag of Words). Целевым классом будет id пользователя.**

### <center>Пример для иллюстрации</center>
**Пусть пользователя всего 2, длина сессии – 2 сайта.**

<center>user1.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">id</th>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:01</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-yw4l">1</td>
    <td class="tg-yw4l">00:00:11</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:16</td>
    <td class="tg-031e">vk.com</td>
  </tr>
  <tr>
    <td class="tg-031e">1</td>
    <td class="tg-031e">00:00:20</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

<center>user2.csv</center>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">id</th>
    <th class="tg-031e">timestamp</th>
    <th class="tg-031e">site</th>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:02</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2</td>
    <td class="tg-yw4l">00:00:14</td>
    <td class="tg-yw4l">google.com</td>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:17</td>
    <td class="tg-031e">facebook.com</td>
  </tr>
  <tr>
    <td class="tg-031e">2</td>
    <td class="tg-031e">00:00:25</td>
    <td class="tg-031e">yandex.ru</td>
  </tr>
</table>

Идем по 1 файлу, нумеруем сайты подряд: vk.com – 1, google.com – 2 и т.д. Далее по второму файлу. 

Отображение сайтов в их индесы должно получиться таким:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-031e">site</th>
    <th class="tg-yw4l">site_id</th>
  </tr>
  <tr>
    <td class="tg-yw4l">vk.com</td>
    <td class="tg-yw4l">1</td>
  </tr>
  <tr>
    <td class="tg-yw4l">google.com</td>
    <td class="tg-yw4l">2</td>
  </tr>
  <tr>
    <td class="tg-yw4l">yandex.ru</td>
    <td class="tg-yw4l">3</td>
  </tr>
  <tr>
    <td class="tg-yw4l">facebook.com</td>
    <td class="tg-yw4l">4</td>
  </tr>
</table>

Тогда обучающая выборка будет такой:
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-s6z2{text-align:center}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-hgcj">session_id</th>
    <th class="tg-hgcj">site1</th>
    <th class="tg-hgcj">site2</th>
    <th class="tg-amwm">target</th>
  </tr>
  <tr>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">2</td>
    <td class="tg-s6z2">1</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">1</td>
  </tr>
  <tr>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-s6z2">2</td>
    <td class="tg-baqh">2</td>
  </tr>
  <tr>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">4</td>
    <td class="tg-s6z2">3</td>
    <td class="tg-baqh">2</td>
  </tr>
</table>

Здесь 1 объект – это сессия из 2 посещенных сайтов 1-ым пользователем (target=1). Это сайты vk.com и google.com (номер 1 и 2). И так далее, всего 4 сессии.

## Часть 1. Подготовка обучающей выборки
Реализуйте функцию *prepare_train_set*, которая принимает на вход путь к каталогу с csv-файлами и параметр *session_length* – длину сессии, а возвращает 2 объекта:
- матрицу (двухмерный NumPy array), в которой строки соответствуют сессиям из *session_length* сайтов, *session_length* столбцов – индексам этих *session_length* сайтов и последний столбец – ID пользователя. 
- частотный словарь сайтов вида {'site_string': (site_id, site_freq)}, например для недавнего игрушечного примера это будет {'vk.com': (1, 2), 'google.com': (2, 2), 'yandex.ru': (3, 3), 'facebook.com': (4, 1)}

Детали:
- Используйте glob (или аналоги) для обхода файлов в каталоге
- Создайте частотный словарь уникальных сайтов (вида {'site_string': (site_id, site_freq)}) и заполняйте его по ходу чтения файлов. Начинайте с 1, то есть первый попавшийся сайт кодируйте единицей
- Не делайте entity recognition, считайте *google.com*, *http://www.google.com* и *www.google.com* разными сайтами
- Скорее всего в файле число записей не кратно числу *session_length*. Тогда последняя сессия будет короче. Остаток заполняйте нулями. То есть если в файле 24 записи и сессии длины 10, то 3 сессия будет состоять из 4 сайтов, и ей мы сопоставим вектор [*site1_id*, *site2_id*, *site3_id*, *site4_id*, 0, 0, 0, 0, 0, 0, *user_id*] 
- 150 файлов из *capstone_websites_data/150users/* должны обрабатываться примерно за 30 секунд, но многое, конечно, зависит от реализации функции и от используемого железа. 

In [7]:
def prepare_train_set(csv_files_mask, session_length):
    '''
    input 
    csv_files_mask = dir of session files,
    session_length = size of the session
    output 
    matrix - array where rows are sessions, columns are indexes of site_id, last column is user_id (target)
    '''   
    from glob import glob 
    
    data = pd.DataFrame(columns = None, dtype = np.int64)
    dic = {} # dictionary where keys are site names values are (site_id, frequency of site_id)
    i = 1 # indicator of sites - site_id
    
    # from csv_files_mask to read all files
    for f in glob(csv_files_mask):
        df = pd.read_csv( f, header=None, names=['id', 'timestamp', 'site']) # make the dataframe from file
        # array with site_id  from dataframe now it is initialed  zeros length is multiple to the session_length
        ar = np.zeros((np.ceil(len(df)/session_length)*session_length, 1), dtype=np.int64) 
        
        
        for j,x in enumerate (df.site):
            # empty dictionary and array 
            if x in dic.keys():
                dic[x] = (dic[x][0], dic[x][1]+1) 
            else:
                dic[x]=(i,1)
                i +=1
            ar[j] = dic[x][0] # 
        
        ar = ar.reshape((np.ceil(len(df)/session_length),session_length)) # convert array to matrix
        
        df1 = pd.DataFrame(ar, columns = None, dtype = np.int64) # convert array to datafarme
        df1['target'] = df['id']  # add target column
        
        data = data.append(df1) # add dataframe from next file  in dir csv_files_mask
        
    return data.values, dic

**Примените полученную функцию к игрушечному примеру, убедитесь, что все работает как надо. Это можно сделать просто глазами (подход eye-balling), а также проверив 2 выражения assert ниже. Если при  выполнении это клетки ничего не произойдет, то скорее функция реализована привильно, а возникновение AssertionError говорит о проблемах с *prepare_train_set*.**

In [8]:
train_data_toy, site_freq_3users = prepare_train_set('capstone_websites_data/3users_toy/*', 
                                                     session_length=10)

In [9]:
train_data_toy

array([[ 1,  2,  2,  3,  2,  4,  5,  6,  7,  8,  1],
       [ 1,  4,  4,  4,  0,  0,  0,  0,  0,  0,  1],
       [ 1,  2,  9,  9,  2,  0,  0,  0,  0,  0,  2],
       [10,  4,  2,  4,  2,  4,  4,  6, 11, 10,  3],
       [10,  4,  2,  0,  0,  0,  0,  0,  0,  0,  3]], dtype=int64)

In [10]:
site_freq_3users

{'accounts.google.com': (5, 1),
 'apis.google.com': (7, 1),
 'football.kulichki.ru': (9, 2),
 'geo.mozilla.org': (3, 1),
 'google.com': (4, 9),
 'mail.google.com': (6, 2),
 'meduza.io': (10, 3),
 'oracle.com': (2, 8),
 'plus.google.com': (8, 1),
 'vk.com': (1, 3),
 'yandex.ru': (11, 1)}

In [11]:
assert train_data_toy.sum() == 159
assert site_freq_3users['meduza.io'] == (10, 3)

**1. Примените полученную функцию к данным по 10 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции write_answer_to_file – это будет ответом к первому вопросу теста. Если ответ получился неправильный, значит где-то ошибка в функции prepare_train_set.**

In [13]:
%time train_data_10users, site_freq_10users = prepare_train_set('capstone_websites_data/10users/*', session_length=10)

Wall time: 15.8 s


In [14]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [15]:
train_data_10users.shape, len(train_data_10users)

((14061L, 11L), 14061)

In [16]:
#write_answer_to_file(len(train_data_10users), 'answer1_1.txt')

**2. Сколько всего уникальных сайтов в выборке из 10 пользователей? **

In [17]:
len(site_freq_10users)

4913

In [18]:
#write_answer_to_file(len(site_freq_10users), 'answer1_2.txt')

**3. Примените полученную функцию к данным по 150 пользователям, посмотрите, сколько уникальных сессий из 10 сайтов получится, запишите ответ в файл с помощью функции write_answer_to_file – это будет ответом к третьему вопросу. Если ответ получился неправильный, значит где-то ошибка в функции prepare_train_set.**

In [19]:

%time train_data_150users, site_freq_150users = prepare_train_set('capstone_websites_data/150users/*.csv',session_length=10)

Wall time: 25min 39s


In [20]:
len(train_data_150users), len(site_freq_150users)

(137019, 27797)

In [21]:
#write_answer_to_file(len(train_data_150users), 'answer1_3.txt')

**4. Сколько всего уникальных сайтов в выборке из 150 пользователей? **

In [22]:
#write_answer_to_file(len(site_freq_150users), 'answer1_4.txt')

**5. Выведите топ-10 самых популярных сайтов среди посещенных 150 пользователями. Запишите ответ в файл *answer1_5.txt* через пробел – все 10 сайтов на одной строке.**

In [22]:
b =site_freq_10users.items() # convert to list

In [23]:
b.sort(key =lambda x: x[1][1], reverse=True) # sort list by frequensy

In [24]:
' '.join([x[0] for x in b[:10]]) # top 10 by frequensy

's.youtube.com www.google.fr www.google.com mail.google.com www.facebook.com apis.google.com r3---sn-gxo5uxg-jqbe.googlevideo.com r1---sn-gxo5uxg-jqbe.googlevideo.com plus.google.com accounts.google.com'

In [25]:
b = site_freq_150users.items()
b.sort(key =lambda x: x[1][1], reverse=True)
top10_popular = [x[0] for x in b[:10]]

In [26]:
top10_popular

['www.google.fr',
 'www.google.com',
 'www.facebook.com',
 'apis.google.com',
 's.youtube.com',
 'clients1.google.com',
 'mail.google.com',
 'plus.google.com',
 'safebrowsing-cache.google.com',
 'www.youtube.com']

In [28]:
#write_answer_to_file(" ".join(top10_popular), 'answer1_5.txt')

In [27]:
" ".join(top10_popular)

'www.google.fr www.google.com www.facebook.com apis.google.com s.youtube.com clients1.google.com mail.google.com plus.google.com safebrowsing-cache.google.com www.youtube.com'

**Для дальнейшего анализа превратим полученные матрицы опять в DataFrame и запишем в csv-файлы.**

In [28]:
train_df_10users = pd.DataFrame(train_data_10users, 
                        columns=['site' + str(i) for i in range(1,11)] + ['target'])
train_df_150users = pd.DataFrame(train_data_150users, 
                        columns=['site' + str(i) for i in range(1,11)] + ['target'])

**Сохраним полученные DataFrame в csv-файлы, при этом учтем, что все признаки у нас целочисленные, поэтому запишем их в соответствующем формате: у DataFrame.to_csv аргумент float_format='%d'. Так на одних только символах разделителя и последующего нуля можно сэкономить почти 30% места.**

In [29]:
train_df_10users.to_csv('capstone_websites_data/train_data_10users.csv', 
                        index_label='session_id', float_format='%d')
train_df_150users.to_csv('capstone_websites_data/train_data_150users.csv', 
                         index_label='session_id', float_format='%d')

## Часть 2. Работа с разреженным форматом данных

**Если так подумать, то полученные признаки site1, ..., site10 смысла не имеют как признаки в задаче классификации. А вот если воспользоваться идеей мешка слов из анализа текстов – это другое дело. Создадим новые матрицы, в которых строкам будут соответствовать сессии из 10 сайтов, а столбцам – индексы сайтов. На пересечении строки $i$ и столбца $j$ будет стоять число $n_{ij}$ – cколько раз сайт $j$ встретился в сессии номер $i$. Делать это будем с помощью разреженных матриц Scipy – [csr_matrix](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html). Прочитайте документацию, разберитесь, как использовать разреженные матрицы и создайте такие матрицы для наших данных. Сначала проверьте на игрушечном примере, затем примените для 10 и 150 пользователей. **

**Обратите внимание, что в коротких сессиях, меньше 10 сайтов, у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.** 

In [12]:
X_toy, y_toy = train_data_toy[:, :-1], train_data_toy[:, -1]

In [13]:
X_toy

array([[ 1,  2,  2,  3,  2,  4,  5,  6,  7,  8],
       [ 1,  4,  4,  4,  0,  0,  0,  0,  0,  0],
       [ 1,  2,  9,  9,  2,  0,  0,  0,  0,  0],
       [10,  4,  2,  4,  2,  4,  4,  6, 11, 10],
       [10,  4,  2,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [14]:
y_toy

array([1, 1, 2, 3, 3], dtype=int64)

In [15]:
def matrix_to_sparse_matrix (matrix):
    """переводим обычную матрицу в разреженноу матрицу 
    где 
    номер столбца это уникальное число из исходной матрицы от 1  до максимального
    значение в строке это сколько раз уникальное число встречалось в строке оригинальной матрицы"""
    from scipy.sparse import csr_matrix
    
    NMZ = np.prod(np.array(matrix.shape)) # колличество элементов в matrix
    data = np.array([1]*NMZ)
    indptr = np.arange(0, NMZ+matrix.shape[1], matrix.shape[1])
    return csr_matrix((data, matrix.reshape(-1), indptr), dtype=int)[:,1:]

In [16]:
X_sparse_toy = matrix_to_sparse_matrix(X_toy)  

In [17]:
X_sparse_toy.todense()

matrix([[1, 3, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0],
        [0, 2, 0, 4, 0, 1, 0, 0, 0, 2, 1],
        [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]])

In [20]:
train_df_10users = pd.read_csv('capstone_websites_data/train_data_10users.csv', index_col='session_id')
train_df_10users.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target
session_id,,,,,,,,,,,
0,1,2,2,3,4,5,6,7,8,7,31
1,7,8,7,7,9,10,11,12,13,14,31
2,7,15,16,17,17,8,17,17,10,3,31
3,18,19,20,17,21,17,17,17,17,22,31
4,23,24,25,17,26,27,28,29,30,31,31


In [26]:
X_10users, y_10users = train_df_10users.ix[:, :-1].values, train_df_10users.ix[:, -1].values
#X_150users, y_150users = train_data_150users[:, :-1], train_data_150users[:, -1]

In [27]:
%time X_sparse_10users =  matrix_to_sparse_matrix(X_10users)

Wall time: 21 ms


In [28]:
train_df_150users = pd.read_csv('capstone_websites_data/train_data_150users.csv', index_col='session_id')
train_df_150users.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target
session_id,,,,,,,,,,,
0,1,2,3,2,3,1,4,5,6,6,6
1,1,1,4,1,1,1,7,1,1,1,6
2,8,1,9,10,9,10,11,8,9,10,6
3,12,13,8,9,12,13,11,8,9,10,6
4,10,12,8,11,9,13,13,12,8,11,6


In [29]:
X_150users, y_150users = train_df_150users.ix[:, :-1].values, train_df_150users.ix[:, -1].values

In [30]:
%time X_sparse_150users =  matrix_to_sparse_matrix(X_150users)

Wall time: 285 ms


**Сохраним эти разреженные матрицы с помощью [pickle](https://docs.python.org/2/library/pickle.html) (сериализация в Python), также сохраним вектора y_10users, y_150users – целевые значения (id пользователя)  в выборках из 10 и 150 пользователей. То что названия этих матриц начинаются с X и y, намекает на то, что на этих данных мы будем проверять первые модели классификации.
Наконец, сохраним также и частотные словари сайтов для 3, 10 и 150 пользователей.**

In [31]:
with open('capstone_websites_data/X_sparse_10users.pkl', 'wb') as X10_pkl:
    pickle.dump(X_sparse_10users, X10_pkl)
with open('capstone_websites_data/y_10users.pkl', 'wb') as y10_pkl:
    pickle.dump(y_10users, y10_pkl)
#with open('capstone_websites_data/site_freq_10users.pkl', 'wb') as site_freq_10users_pkl:
#    pickle.dump(site_freq_10users, site_freq_10users_pkl)    
with open('capstone_websites_data/X_sparse_150users.pkl', 'wb') as X150_pkl:
    pickle.dump(X_sparse_150users, X150_pkl)
with open('capstone_websites_data/y_150users.pkl', 'wb') as y150_pkl:
    pickle.dump(y_150users, y150_pkl)
#with open('capstone_websites_data/site_freq_150users.pkl', 'wb') as site_freq_150users_pkl:
#    pickle.dump(site_freq_150users, site_freq_150users_pkl)

**Убедимся, что число столбцов в разреженных матрицах X_sparse_10users и X_sparse_150users равно ранее посчитанным числам уникальных сайтов для 10 и 150 пользователей соответственно. Важно удостовериться, что потом мы все будем работать с одними и теми же данными.**

In [41]:
assert X_sparse_10users.shape[1] == len(site_freq_10users)

In [42]:
X_sparse_10users.shape, len(site_freq_10users)

((14061, 4913), 4913)

In [43]:
assert X_sparse_150users.shape[1] == len(site_freq_150users)

In [44]:
X_sparse_150users.shape

(137019, 27797)

## Пути улучшения
В этом проекте свобода творчества на каждом шаге, а 6 и 7 недели проекта посвящены общему описанию проекта (.ipynb или pdf) и взаимному оцениванию. Что еще можно добавить по первой части проекта:
- при наличии доступа к хорошим мощностям (можно арендовать инстанс Amazon EC2, как именно, описано [тут](https://habrahabr.ru/post/280562/)) можно обработать исходные данные по 3000 пользователей